In [ ]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
from eo_tools.S1.core import S1IWSwath, align, coregister
from eo_tools.S1.util import presum, boxcar
from eo_tools.util import show_cog
import numpy as np
import logging
logging.basicConfig(level=logging.INFO)
import matplotlib.pyplot as plt
import numpy as np
import rasterio

from folium import LayerControl
# import xarray as xr

import logging

log = logging.getLogger(__name__)

In [ ]:
data_dir = "/data/S1"
primary_dir = f"{data_dir}/S1A_IW_SLC__1SDV_20230903T183344_20230903T183412_050167_0609B4_100E.SAFE"
secondary_dir = f"{data_dir}/S1A_IW_SLC__1SDV_20230915T183345_20230915T183413_050342_060F9F_85A4.SAFE"
out_dir = f"/data/res/test_new_proc"
iw = 1
pol = "vv"
up = 2

burst_min = 1
burst_max = 2

In [ ]:
# import rasterio
# with rasterio.open("/data/res/test_new_proc/primary.tif") as ds:
#     img = ds.read(1)
# with rasterio.open("/data/res/test_new_proc/secondary.tif") as ds:
#     img2 = ds.read(1)

# plt.figure(figsize=(10, 10))
# plt.imshow(np.angle(img*img2.conj())[:,::8].T, interpolation="none", vmax = 1)
# plt.colorbar(fraction=0.046, pad=0.04)

In [ ]:
from eo_tools.S1.process import preprocess_insar_iw
preprocess_insar_iw(primary_dir, secondary_dir, out_dir, burst_min, burst_max)

In [ ]:
from eo_tools.S1.process import slc2geo, interferogram, coherence
file_prm = f"{out_dir}/primary.tif"
file_sec = f"{out_dir}/secondary.tif"
file_ifg = f"{out_dir}/ifg_rdr.tif"
file_phi = f"{out_dir}/phi_geo.tif"
file_coh_r = f"{out_dir}/coh_rdr.tif"
file_coh = f"{out_dir}/coh_geo.tif"
file_lut = f"{out_dir}/lut.tif"
interferogram(file_prm, file_sec, file_ifg)
coherence(file_prm, file_sec, file_coh_r, box_size=5, magnitude=True)
slc2geo(file_ifg, file_lut, file_phi, 2, 8, 3, True)
slc2geo(file_coh_r, file_lut, file_coh, 2, 8, 3, False)

In [ ]:
import xarray as xr
coh = xr.open_dataset(file_coh, engine="rasterio")
plt.figure(figsize=(10, 10))
plt.imshow(coh["band_data"][0], interpolation="none")
plt.colorbar(fraction=0.046, pad=0.04)

In [ ]:
import xarray as xr
phi = xr.open_dataset(file_phi, engine="rasterio")
plt.figure(figsize=(10, 10))
plt.imshow(phi["band_data"][0], interpolation="none")
plt.colorbar(fraction=0.046, pad=0.04)

In [ ]:
def make_da_from_dem(arr, file_dem):

    from rasterio.enums import Resampling

    dem_ds = xr.open_dataset(file_dem, engine="rasterio")
    # new_width = dem_ds.rio.width * up
    # new_height = dem_ds.rio.height * up
    new_width = arr.shape[1]
    new_height = arr.shape[0]

    # upsample dem
    dem_up_ds = dem_ds.rio.reproject(
        dem_ds.rio.crs,
        shape=(int(new_height), int(new_width)),
        resampling=Resampling.bilinear,
    )
    da = xr.DataArray(
        data=arr[None],
        coords=dem_up_ds.coords,
        dims=("band", "y", "x"),
        attrs=dem_up_ds.attrs,
    )
    da.attrs['_FillValue'] = np.nan
    return da

In [ ]:
# da = make_da_from_dem(az_p2g, file_dem, 2)
# dem_ds.spatial_ref.GeoTransform, 

In [ ]:
prm = S1IWSwath(master_dir, iw=iw, pol=pol)
sec = S1IWSwath(slave_dir, iw=iw, pol=pol)
overlap = np.round(prm.compute_burst_overlap(2)).astype(int)

lut_az = []
lut_rg = []

for burst_idx in range(1, 3):
    log.info(f"---- Processing burst {burst_idx} ----")

    # compute geocoding LUTs for master and slave bursts
    file_dem = prm.fetch_dem_burst(burst_idx, dir_dem="/data/tmp", force_download=False)
    az_p2g, rg_p2g, _ = prm.geocode_burst(
        file_dem, burst_idx=burst_idx, dem_upsampling=up
    )
    az_s2g, rg_s2g, _ = sec.geocode_burst(
        file_dem, burst_idx=burst_idx, dem_upsampling=up
    )

    nl, nc = az_p2g.shape

    # read primary and secondary burst raster
    arr_p = prm.read_burst(burst_idx, True)
    arr_s = sec.read_burst(burst_idx, True)

    # deramp secondary
    pdb_s = sec.deramp_burst(burst_idx)
    arr_s_de = arr_s * np.exp(1j * pdb_s).astype(np.complex64)

    # project slave LUT into master grid
    az_s2p, rg_s2p = coregister(arr_p, az_p2g, rg_p2g, az_s2g, rg_s2g)

    # warp raster slave and deramping phase
    arr_s2p = align(arr_p, arr_s_de, az_s2p, rg_s2p, order=3)
    pdb_s2p = align(arr_p, pdb_s, az_s2p, rg_s2p, order=3)

    # reramp slave
    arr_s2p = arr_s2p * np.exp(-1j * pdb_s2p).astype(np.complex64)

    # compute topographic phases
    rg_p = np.zeros(arr_s.shape[0])[:, None] + np.arange(0, arr_s.shape[1])
    pht_p = prm.phi_topo(rg_p).reshape(*arr_p.shape)
    pht_s = sec.phi_topo(rg_s2p.ravel()).reshape(*arr_p.shape)
    pha_topo = np.exp(-1j * (pht_p - pht_s)).astype(np.complex64)

    az_da = make_da_from_dem(az_p2g, file_dem)
    rg_da = make_da_from_dem(rg_p2g, file_dem)
    lut_az.append(az_da)
    lut_rg.append(rg_da)

    arr_s2p = arr_s2p * pha_topo


# required:
# slc stack made of all arr_p and arr_s2p

# interferogram without topographic phase
# ifg = (arr_s2p * arr_p.conj() * pha_topo).conj()

# normalize complex coherences
# pows = avg_ampl(arr_p, boxsiz) * avg_ampl(arr_s2p, boxsiz)
# ifgs.append(boxcar(np.nan_to_num(ifg), boxsiz, boxsiz) / pows)
# lut.append((az_p2g, rg_p2g))
# dems.append(file_dem)

In [ ]:
# from eo_tools.S1_dev import fast_esd
# fast_esd(ifgs, overlap)

In [ ]:
# from eo_tools.S1_dev import stitch_bursts
# img = stitch_bursts(ifgs, overlap)

In [ ]:
from eo_tools.S1.core import resample
from eo_tools.util import palette_phi
from folium import Map
import rioxarray as riox
from rioxarray.merge import merge_arrays
# xr.set_options(keep_attrs=True)

def merge_luts(luts_az, luts_rg):
    off = 0
    H = int(overlap / 2)
    # phi_out = presum(np.nan_to_num(img), 2, 8)
    naz = prm.lines_per_burst
    to_merge_az = []
    to_merge_rg = []
    for i in range(len(lut_az)):
        az_mst, rg_mst = luts_az[i].copy(), luts_rg[i].copy()
        cnd = (az_mst >= H - 4) & (az_mst < naz - H + 4)
        az_mst.data[~cnd] = np.nan
        rg_mst.data[~cnd] = np.nan

        # does the job but not very elegant
        if i == 0:
            off2 = off
        else:
            off2 = off - H
        az_mst = az_mst + off2
        if i == 0:
            off += naz - H
        else:
            off += naz - 2 * H
        to_merge_az.append(az_mst)
        to_merge_rg.append(rg_mst)

    merged_az = merge_arrays(to_merge_az)
    merged_rg = merge_arrays(to_merge_rg)
    merged_az.rio.to_raster("/data/res/test_merge_az.tif")
    merged_rg.rio.to_raster("/data/res/test_merge_rg.tif")

In [ ]:
# lut_az[0].rio.to_raster("/data/res/test_georef_az.tif")

In [ ]:
# plt.figure(figsize=(10, 10))
# plt.imshow(az_mst[0], interpolation="none")
# plt.colorbar(fraction=0.046, pad=0.04)

In [ ]:
import folium
m = Map()
tile = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = False,
        control = True
       ).add_to(m)

# hack: avoid titiler caching the tiles
rnd = np.random.rand() * 1e-6

show_cog("/data/res/test_merge_az.tif", m,rescale=f"0,{3002+rnd}")
LayerControl().add_to(m)
m

In [ ]:
from eo_tools.S1_dev import resample
from eo_tools.util import palette_phi
from folium import Map
import rioxarray as riox
from rioxarray.merge import merge_arrays

off = 0
H = int(overlap / 2)
phi_out = presum(np.nan_to_num(img), 2, 8)
naz = ifgs[0].shape[0]
list_ifg = []
for i in range(len(ifgs)):
    az_mst, rg_mst = lut[i]
    file_dem = dems[i]
    cnd = (az_mst >= H - 4) & (az_mst < naz - H + 4)
    az_mst2 = az_mst.copy()
    rg_mst2 = rg_mst.copy()
    az_mst2[~cnd] = np.nan
    rg_mst2[~cnd] = np.nan

    file_ifg = f"/data/res/test_remap_burst_{i}_ifg.tif"

    # does the job but not very elegant
    if i == 0:
        off2 = off
    else:
        off2 = off - H
    resample(
        phi_out,
        file_dem,
        file_ifg,
        (az_mst2 + off2) / 2,
        (rg_mst2) / 8,
        order=3,
    )
    if i == 0:
        off += naz - H
    else:
        off += naz - 2 * H

    list_ifg.append(riox.open_rasterio(file_ifg))

merged_ifg = merge_arrays(list_ifg)
merged_ifg.rio.to_raster("/data/res/test_merge_ifg.tif")

In [ ]:
# compute phase and coherence

# avoid metadata being lost in arithmetic opetations
xr.set_options(keep_attrs=True)
ifg = riox.open_rasterio("/data/res/test_merge_ifg.tif")
phi = np.arctan2(ifg[1], ifg[0])
img = np.sum(np.square(ifg),0)
phi.rio.to_raster("/data/res/test_merge_phi2.tif", nodata=0)
img.rio.to_raster("/data/res/test_merge_coh2.tif", nodata=0)

In [ ]:
import folium
m = Map()
tile = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = False,
        control = True
       ).add_to(m)

# hack: avoid titiler caching the tiles
scale_phi = np.pi + np.random.rand() * 1e-6
rnd_coh = np.random.rand() * 1e-6

show_cog("/data/res/test_merge_coh2.tif", m,rescale=f"0,{1+rnd_coh}")
show_cog("/data/res/test_merge_phi2.tif", m,rescale=f"-{scale_phi}, {scale_phi}",colormap=palette_phi())
LayerControl().add_to(m)
m

In [ ]:
file_dem

In [ ]:
# xr.open_dataset("/data/res/pre_processed.nc")
# Open the existing NetCDF file in append mode
# with xr.open_dataset('existing_file.nc', mode='a') as ds:
    # Assuming `raster_data` is the raster data you want to write
    # Assuming `desired_date` is the specific date where you want to write the raster

    # Select the specific date and assign the raster data
    # ds['raster_primary'].loc[desired_date] = raster_data

    # Write the modified dataset back to the NetCDF file without loading it into memory
    # ds.to_netcdf('existing_file.nc', mode='a', compute=False)


In [ ]:
import numpy as np
np.array(2.0, dtype="float32")